In [1]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# % matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Importing libraries for building the neural network
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

from sklearn.utils import shuffle
from keras.optimizers import SGD
from sklearn.preprocessing import StandardScaler

from numpy import array
from numpy import argmax
from keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn import svm

cnf_list = []
number_of_neurons = int(input("Podaj liczbę neuronów: "))
momentum_value = float(input("Podaj wartość momentum, wartość zero oznacza brak momentu, dopuszczalne liczby zmiennoprzecinkowe: "))
for counter in range(5):
    excel_data = pd.read_excel('data/oczyszczone.xls', sheet_name = 'daneSOB')
    headers = excel_data.columns.values
    excel_data.head()
    excel_data.groupby('Klasy diagnozy').size()
    diagnosis_classes = len(excel_data.groupby('Klasy diagnozy').size())
    diagnosis_classes = [ str(x + 1) for x in range(diagnosis_classes)]
    diagnosis_classes
    excel_data = shuffle(excel_data)
    # one hot encoding
    for i in headers:
        le = LabelEncoder()
        le.fit(excel_data[str(i)])
        excel_data[str(i)] = le.transform(excel_data[str(i)])
        encoded = to_categorical(excel_data[str(i)])

        if len(encoded[0]) > 2 and i != 'Klasy diagnozy':
            insert_loc = np.where(headers==i)
            insert_loc = insert_loc[0][0]

            excel_data = excel_data.drop(columns = i)
            for label in range(len(encoded[0])):
                excel_data.insert(int(insert_loc), str(i) + "_" + str(label), encoded[:,label])


    excel_data.head()
    Y_data = excel_data['Klasy diagnozy']
    X_data = excel_data.drop(columns = 'Klasy diagnozy')
     # ratio 1/2 to 1/2
    X_train = X_data[:238]
    Y_train = Y_data[:238]

    X_test = X_data[238:]
    Y_test = Y_data[238:]

    kfold = [[X_train, Y_train, X_test, Y_test], [ X_test, Y_test, X_train, Y_train]]
    cvscores = []

   
    model = Sequential()
    model.add(Dense(units = 106,activation = 'relu',input_dim = 106)) # warstwa wejściowa
    model.add(Dense(units = number_of_neurons, activation = 'relu')) # warstwa ukryta
    model.add(Dense(units = 8, activation = 'softmax')) # warstwa wyjściowa
    for X_train, Y_train, X_test, Y_test in kfold:


        model.compile(optimizer= SGD(lr = 0.07, momentum=momentum_value),loss= 'sparse_categorical_crossentropy', metrics= ['accuracy'])
        model.fit(X_train, Y_train, epochs = 30,shuffle = True, verbose = 0, validation_data=(X_test, Y_test))
        scores = model.evaluate(X_test, Y_test, verbose=0)
        print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
        cvscores.append(scores[1] * 100)
    print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
    classifier = svm.SVC(kernel='linear', C=0.04)
    y_pred = classifier.fit(X_train, Y_train).predict(X_test)
    cnf_matrix = confusion_matrix(Y_test, y_pred)
    cnf_list.append(cnf_matrix)
    print(cnf_matrix)
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = np.round(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], 2)
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

cnf = 0
for x in cnf_list:
    cnf = cnf + x
cnf = cnf/5
    
print(cnf)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf, classes=diagnosis_classes,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf, classes=diagnosis_classes, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

Using TensorFlow backend.


Podaj liczbę neuronów: 106
Podaj wartość momentum, wartość zero oznacza brak momentu, dopuszczalne liczby zmiennoprzecinkowe: 1
acc: 29.83%
acc: 29.41%
29.62% (+/- 0.21%)
[[70  0  0  0  0  0  0  0]
 [ 9  0  0  0  0  0  0  0]
 [ 0  0 17  0  0  0  0  0]
 [ 1  0  0 11  0  0  0  0]
 [ 0  0  0  0 29  0  0  0]
 [ 1  0  0  0  0 12  2  0]
 [ 0  0  0  0  0  0 78  0]
 [ 0  0  0  0  0  0  0  8]]
acc: 30.25%
acc: 35.71%
32.98% (+/- 2.73%)
[[73  0  0  0  0  0  0  0]
 [ 7  0  0  0  0  0  1  0]
 [ 0  0 16  0  0  0  0  0]
 [ 0  0  0 15  0  0  0  0]
 [ 0  0  0  0 24  0  0  0]
 [ 0  0  0  0  0 11  0  0]
 [ 0  0  0  0  0  0 85  0]
 [ 0  0  0  0  0  0  0  6]]
acc: 9.66%
acc: 17.23%
13.45% (+/- 3.78%)
[[68  0  0  1  0  0  1  0]
 [ 5  1  0  0  0  0  2  0]
 [ 0  0 17  0  0  0  0  0]
 [ 0  0  0 11  0  0  0  0]
 [ 1  0  0  0 35  0  0  0]
 [ 0  0  0  0  0 14  0  0]
 [ 0  0  0  0  0  0 71  0]
 [ 1  0  0  0  0  0  2  8]]
acc: 31.09%
acc: 34.87%
32.98% (+/- 1.89%)
[[74  0  0  0  0  0  0  0]
 [ 7  1  0  0  0  0  2 